In [ ]:
import pandas as pd
import os
import numpy as np
from gensim.models import Word2Vec

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load data
new_notes = pd.read_pickle("data/ner_df.p")
w2vec = Word2Vec.load("embeddings/word2vec.model")

In [ ]:
# Filter out rows with empty 'ner' values
null_index_list = [i.Index for i in new_notes.itertuples() if len(i.ner) == 0]
new_notes.drop(null_index_list, inplace=True)

In [ ]:
# Process NER data
med7_ner_data = {}
for ii in new_notes.itertuples():
    p_id = ii.SUBJECT_ID
    ind = ii.Index

    try:
        new_ner = new_notes.loc[ind].ner
    except:
        new_ner = []

    new_temp = [(k[0], k[1]) for j in new_ner for k in j]

    if p_id in med7_ner_data:
        med7_ner_data[p_id].extend(new_temp)
    else:
        med7_ner_data[p_id] = new_temp


In [ ]:
# Save processed NER data
pd.to_pickle(med7_ner_data, "data/new_ner_word_dict.pkl")

In [ ]:
# Calculate mean for vectors
def mean(a):
    return sum(a) / len(a)

In [ ]:
# Process Word2Vec embeddings
data_types = [med7_ner_data]
data_names = ["new_ner"]

for data, names in zip(data_types, data_names):
    new_word2vec = {}
    print("w2vec starting..")
    for k, v in data.items():
        patient_temp = []
        if isinstance(v, list):
            for i in v:
                if isinstance(i, tuple) and len(i) == 2 and isinstance(i[0], str):
                    if i[0] in w2vec.wv:
                        patient_temp.append(w2vec.wv[i[0]])
                    elif len(i[0].split(" ")) > 1:
                        avg = []
                        words = i[0].split(" ")
                        num = 0
                        for each_word in words:
                            if each_word in w2vec.wv:
                                temp = w2vec.wv[each_word]
                                avg.append(temp)
                                num += 1
                        if num > 0:
                            avg = np.asarray(avg)
                            t = np.asarray(list(map(mean, zip(*avg))))
                            patient_temp.append(t)
        if patient_temp:
            new_word2vec[k] = patient_temp

    print(len(new_word2vec))
    pd.to_pickle(new_word2vec, "data/"+names+"_word2vec_dict.pkl")